# Coupling BEM-BEM-BEM

<img src="2_Interfaces_BBB_EG.png" width="500" height="250">

To calculate the solvation energy of a solute using the BEM-BEM-BEM coupling, the following matrix system must be solved, which comes from solving the following PDE, including its border conditions:
\begin{equation} 
\left\{\begin{matrix} 
     -\epsilon_{m}\nabla^{2} \phi_{m}(x)=\sum_{i=1}^{n_{c}}Q_{i}\delta\left (x-x_{i} \right ) & x \ \in \ \Omega_{m} \\
     -\nabla^{2}\phi_{i}(x)+\kappa_{i}^{2}\phi_{i}(x)
     =0 \ & x \ \in \ \Omega_{i} \\ 
     -\nabla^{2}\phi_{s}(x)+\kappa_{s}^{2}\phi_{s}(x)=0 & x \ \in \ \Omega_{s} \\
     \phi_{m}(x)= \phi_{i}(x) & x \ \in \ \Gamma_{a} \\
     \epsilon_{m}\partial_n^a \phi_{m}(x)= \epsilon_{i}\partial_n^a \phi_{i}(x) & x \ \in \ \Gamma_{a} \\
     \phi_{i}(x)= \phi_{s}(x) & x \ \in \ \Gamma_{b} \\
     \epsilon_{i}\partial_n^b \phi_{i}(x)= \epsilon_{s}\partial_n^b \phi_{s}(x) & x \ \in \ \Gamma_{b}
\end{matrix}\right. 
\end{equation}

Where $\Omega_{m}$ is the solute domain, $\Omega_{i}$ is the Stern Layer, $\Omega_{s}$ is the solvent domain, $\Gamma_{a}$ is the inner interface and $\Gamma_{b}$ is the outer interface. And its matrix system corresponds to:
\begin{equation}
\begin{bmatrix}
\frac{1}{2}I+K_{L,a}^{a} & -V_{L,a}^{a}  & 0 & 0\\ 
\frac{1}{2}I-K_{I,a}^{a} & \frac{\epsilon_{m}}{\epsilon_{i}}V_{I,a}^{a} &  K_{I,b}^{a} & -V_{I,b}^{a}\\ 
-K_{I,a}^{b} & \frac{\epsilon_{m}}{\epsilon_{i}}V_{I,a}^{b} &  \frac{1}{2}I+K_{I,b}^{b} & -V_{I,b}^{b}\\  
0 & 0 &  \frac{1}{2}I-K_{H,b}^{b} & \frac{\epsilon_{i}}{\epsilon_{s}}V_{H,b}^{b} 
\end{bmatrix} \cdot
\begin{bmatrix}
\phi_{m}^{\Gamma_{a}} \\
\partial_n \phi_{m}^{\Gamma_{a}}\\
\phi_{i}^{\Gamma_{b}} \\
\partial_n \phi_{i}^{\Gamma_{b}}\\ 
\end{bmatrix} =
\begin{bmatrix}
\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma_{a}}-x_{i}\right \|} \\
0\\
0 \\
0 \\ 
\end{bmatrix} 
\end{equation}

And when solving the matrix system, the solvation energy is calculated as:
\begin{equation} 
    \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i} \left ( V_L\partial_n \phi_m^{\Gamma_{a}}(x_{i})-K_L\phi_m^{\Gamma_{a}}(x_{i}) \right )
\end{equation}   

## Implementation

First we import the main libraries such as Numpy, Trimesh and Bempp, as well as files in Python to read the solute files, which are in ".pqr" and ".off" format.

In [1]:
import bempp.api
import numpy as np 
import time
import trimesh
from readoff import *
from readpqr import *
start = time.time()

Next, the physical conditions of the problem in each region are defined, such as pemitivity and
Debye-Huckel parameter.

In [2]:
em = 4.           #[-] Interior electrical permittivity of the solute.
es = 80.          #[-] Exterior electrical permittivity of the solvent.
ei = (es+em)/2    #[-] Intermediate electrical permittivity of the intermediate domain.
ks = 0.125        #[1/A] Inverse of the Debye-Huckel length of the fluid in the solvent.
ki = ks*np.sqrt(es/(es+em))  #[1/A] Inverse of the Debye-Huckel length of the fluid in the intermediate domain.

Later, we choose the solution that we want to use to calculate the energy, where we import the surface mesh and the ".pqr" file to obtain the information on the radii, charges and position of the solute atoms.

In [3]:
Mesh1 ='Sphere/Mallas_S/Sphere5R4.off'  #Path of the inner surface mesh of the solute for which the energy is to be calculated. In "off" format.
Mesh2 ='Sphere/Mallas_S/Sphere8R4.off'  #Path of the outer surface mesh of the solute for which the energy is to be calculated. In "off" format.
PQR = 'PQR/Sphere5Q3.pqr'        #Path of the position and charges of the solute for which the energy is to be calculated. In "pqr" format.
PC,Q,R = readpqr(PQR)   

In addition, we can choose the type of preconditioner to solve the matrix system, the type of assembly of the edge operators, parameters of the GMRES and the FMM if required.

In [4]:
#Choice to solve the matrix equation in BEM/BEM/BEM.
#True: Use the Mass Matrix preconditioner.
#False: Use the Block diagonal preconditioner.
SF = False  #Strong form. 
    
#Assembly of border operators.
#fmm: For molecules with a greater number of vertices.
#default_nonlocal: For molecules with a small number of vertices.    
Assemble = 'default_nonlocal' 

#Important parameters of the GMRES.
Tol =1e-6    #GMRES tolerance.
Res =70      #Restart of the GMRES in each iteration.

#Secondary parameters when working with the fmm assembly.
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5  
bempp.api.GLOBAL_PARAMETERS.fmm.ncrit = 400          
bempp.api.GLOBAL_PARAMETERS.quadrature.regular = 4  

Next, we generate the surface mesh of the solute.

In [5]:
vertices_0,faces_0 = read_off(Mesh1) 
#In case the mesh has small gaps, with trimesh the information of the original mesh without the gaps is obtained.
meshSP = trimesh.Trimesh(vertices = vertices_0, faces= faces_0) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split)) #1 mesh means no cavity.

vertices_1 = mesh_split[0].vertices 
faces_1 = mesh_split[0].faces   
grid1 = bempp.api.grid.grid.Grid(vertices_1.transpose(), faces_1.transpose()) #Creation of the inner surface mesh.

vertices_2 ,faces_2 =read_off(Mesh2) 
grid2 = bempp.api.grid.grid.Grid(vertices_2.transpose(), faces_2.transpose()) #Creation of the outer surface mesh.

Found 1 meshes


We make the Bempp function spaces of the potential and its derivative.

In [6]:
dirichl_space1 = bempp.api.function_space(grid1, "P", 1)  #Electrostatic potential at the inner interface.
neumann_space1 = bempp.api.function_space(grid1, "P", 1)  #Derived from the electrostatic potential at the inner interface. 
dirichl_space2 = bempp.api.function_space(grid2, "P", 1)  #Electrostatic potential at the outer interface.
neumann_space2 = bempp.api.function_space(grid2, "P", 1)  #Derived from the electrostatic potential at the outer interface. 
print("DS1 dofs: {0}".format(dirichl_space1.global_dof_count))
print("NS1 dofs: {0}".format(neumann_space1.global_dof_count))
print("DS2 dofs: {0}".format(dirichl_space2.global_dof_count))
print("NS2 dofs: {0}".format(neumann_space2.global_dof_count))

DS1 dofs: 4591
NS1 dofs: 4591
DS2 dofs: 11924
NS2 dofs: 11924


We create the boundary operators that we need for each domain.

In [7]:
#Identity operators.
I1d = bempp.api.operators.boundary.sparse.identity(dirichl_space1, dirichl_space1, dirichl_space1) # 1
I1n = bempp.api.operators.boundary.sparse.identity(dirichl_space1, neumann_space1, neumann_space1) # 1
I2d = bempp.api.operators.boundary.sparse.identity(dirichl_space2, dirichl_space2, dirichl_space2) # 1
I2n = bempp.api.operators.boundary.sparse.identity(dirichl_space2, neumann_space2, neumann_space2) # 1
#Domain of the solute Ωm.
KLaa = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #K
VLaa = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #V
Z1ba = bempp.api.ZeroBoundaryOperator(dirichl_space2, dirichl_space1, dirichl_space1) #0
Z2ba = bempp.api.ZeroBoundaryOperator(neumann_space2, dirichl_space1, dirichl_space1) #0
#Intermediate domain Ωi at the inner interface.
if ki==0:
    KIaa = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, neumann_space1, neumann_space1, assembler=Assemble) #K
    VIaa = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, neumann_space1, neumann_space1, assembler=Assemble) #V  
    KIba = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space1, neumann_space1, assembler=Assemble) #K
    VIba = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space1, neumann_space1, assembler=Assemble) #V
else:
    KIaa = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space1, neumann_space1, neumann_space1, ki, assembler=Assemble) #K
    VIaa = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space1, neumann_space1, neumann_space1, ki, assembler=Assemble) #V  
    KIba = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, neumann_space1, neumann_space1, ki, assembler=Assemble) #K
    VIba = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, neumann_space1, neumann_space1, ki, assembler=Assemble) #V
#Intermediate domain Ωi at the outer interface.
if ki==0:
    KIab = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #K
    VIab = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #V
    KIbb = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #K
    VIbb = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #V
else:
    KIab = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space1, dirichl_space2, dirichl_space2, ki,  assembler=Assemble) #K
    VIab = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space1, dirichl_space2, dirichl_space2, ki, assembler=Assemble) #V
    KIbb = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, ki, assembler=Assemble) #K
    VIbb = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, dirichl_space2, dirichl_space2, ki, assembler=Assemble) #V     
#Domain of the solvent Ωs.
Z1ab = bempp.api.ZeroBoundaryOperator(dirichl_space1, neumann_space2, neumann_space2) #0
Z2ab = bempp.api.ZeroBoundaryOperator(neumann_space1, neumann_space2, neumann_space2) #0
if ks==0:
    KHbb = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space2, neumann_space2, assembler=Assemble) #K
    VHbb = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space2, neumann_space2, assembler=Assemble) #V
else:
    KHbb = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, neumann_space2, neumann_space2, ks, assembler=Assemble) #K
    VHbb = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, neumann_space2, neumann_space2, ks, assembler=Assemble) #V

Later, we create the Coulomb potential function $\phi_{c}^{\Gamma_{a}}$ and with that, we construct the vector right head side $b$ of formulation.
\begin{equation}
\phi_{c}^{\Gamma_{a}} =\frac{1}{4\pi\epsilon_{m}}\sum_{n=1}^{n_{c}}\frac{Q_{i}}{\left \|x_{\Gamma_{a}}-x_{i}\right \|} \quad \quad
b=\begin{bmatrix}
\phi_{c}^{\Gamma_{a}} \\
0 \\
0 \\
0 \\
\end{bmatrix}
\end{equation}

In [8]:
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uc1 = bempp.api.GridFunction(dirichl_space1, fun=U_c)  

if SF==False:
    # Rhs in Ωm.
    rhs_M = (Uc1).projections(dirichl_space1) 
    # Rhs in Ωi at inner interface.
    rhs_I1 = np.zeros(neumann_space1.global_dof_count) 
    # Rhs in Ωi at outer interface.
    rhs_I2 = np.zeros(dirichl_space2.global_dof_count) 
    # Rhs in Ωs.
    rhs_S = np.zeros(neumann_space2.global_dof_count) 
    # The combination of Rhs.
    rhs = np.concatenate([rhs_M, rhs_I1, rhs_I2, rhs_S])
else:
    Uc2 = bempp.api.GridFunction(dirichl_space2, fun=U_c) 
    rhs = [I1d*Uc1, 0*I1n*Uc1, 0*I2d*Uc2, 0*I2n*Uc2] 

And we construct the global left 4x4 matrix $A$ of the formulation.   
\begin{equation}
A=\begin{bmatrix}
\frac{1}{2}I+K_{L,a}^{a} & -V_{L,a}^{a}  & 0 & 0\\ 
\frac{1}{2}I-K_{I,a}^{a} & \frac{\epsilon_{m}}{\epsilon_{i}}V_{I,a}^{a} &  K_{I,b}^{a} & -V_{I,b}^{a}\\ 
-K_{I,a}^{b} & \frac{\epsilon_{m}}{\epsilon_{i}}V_{I,a}^{b} &  \frac{1}{2}I+K_{I,b}^{b} & -V_{I,b}^{b}\\  
0 & 0 &  \frac{1}{2}I-K_{H,b}^{b} & \frac{\epsilon_{i}}{\epsilon_{s}}V_{H,b}^{b} 
\end{bmatrix}
\end{equation}

In [9]:
if SF==False:
    #Position of the 4x4 matrix.
    blocks = [[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]] 
    blocks[0][0] = (0.5*I1d+KLaa).weak_form()   
    blocks[0][1] = -VLaa.weak_form()           
    blocks[0][2] = Z1ba.weak_form()            
    blocks[0][3] = Z2ba.weak_form()    
    
    blocks[1][0] = (0.5*I1n-KIaa).weak_form()  
    blocks[1][1] = (em/ei)*VIaa.weak_form()    
    blocks[1][2] = KIba.weak_form()            
    blocks[1][3] = -VIba.weak_form()    
    
    blocks[2][0] = -KIab.weak_form()           
    blocks[2][1] = (em/ei)*VIab.weak_form()    
    blocks[2][2] = (0.5*I2d+KIbb).weak_form()  
    blocks[2][3] = -VIbb.weak_form()    
    
    blocks[3][0] = Z1ab.weak_form()            
    blocks[3][1] = Z2ab.weak_form()            
    blocks[3][2] = (0.5*I2n-KHbb).weak_form()  
    blocks[3][3] = (ei/es)*VHbb.weak_form()    
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks)) 
    #Block diagonal preconditioner for BEM.
    from preconditioners import *
    P = BlockDiagonal_4x4(dirichl_space1, neumann_space1, dirichl_space2, neumann_space2, blocks, es,ei,em,ks,ki)
else:
    blocks = bempp.api.BlockedOperator(4,4)   
    #Position of the 4x4 matrix.
    blocks[0,0] = (0.5*I1d+KLaa)  
    blocks[0,1] = -VLaa          
    blocks[0,2] = Z1ba           
    blocks[0,3] = Z2ba       
    
    blocks[1,0] = (0.5*I1n-KIaa) 
    blocks[1,1] = (em/ei)*VIaa   
    blocks[1,2] = KIba           
    blocks[1,3] = -VIba     
    
    blocks[2,0] = -KIab          
    blocks[2,1] = (em/ei)*VIab   
    blocks[2,2] = (0.5*I2d+KIbb) 
    blocks[2,3] = -VIbb      
    
    blocks[3,0] = Z1ab           
    blocks[3,1] = Z2ab           
    blocks[3,2] = (0.5*I2n-KHbb) 
    blocks[3,3] = (ei/es)*VHbb   

Later, we solve the matrix system $Ax=b$ with GMRES together with its respective preconditioning an efficient solution. The solution is then divided into the parts associated with $\phi_{m}^{\Gamma_{a}}$, $\partial_{n}\phi_{m}^{\Gamma_{a}}$, $\phi_{i}^{\Gamma_{b}}$ and $\partial_{n}\phi_{i}^{\Gamma_{b}}$.

In [10]:
#Iteration counter.
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 100) == (it_count // 100):
        print(it_count,x)    
        
# Solution by GMRES.
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=Tol, restart=Res)  
    end1 = time.time() 
else:
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True,tol=Tol, restart=Res)  
    end1 = time.time() 

# Time to solve the equation.
curr_time1 = (end1 - start1)
print("Number of GMRES iterations: {0}".format(it_count))
print("Total time in GMRES: {:5.2f} [s]".format(curr_time1))   

Number of GMRES iterations: 49
Total time in GMRES: 54.33 [s]


Next, we make Bempp functions from the solution of $\phi_{m}^{\Gamma}$ and $\partial_{n}\phi_{m}^{\Gamma}$.

In [11]:
if SF==False:   
    soln_u1  = soln[:dirichl_space1.global_dof_count]
    soln_du1 = soln[dirichl_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count]
    soln_u2  = soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count]
    soln_du2 = soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count:]  
    # Solution for Dirichlet data at inner surface.
    dirichlet_fun1 = bempp.api.GridFunction(dirichl_space1, coefficients=soln_u1)
    # Solution for Neumann data at inner surface.
    neumann_fun1 = bempp.api.GridFunction(neumann_space1, coefficients=soln_du1)
    # Solution for Dirichlet data at outer surface.
    dirichlet_fun2 = bempp.api.GridFunction(dirichl_space2, coefficients=soln_u2)
    # Solution for Neumann data at outer surface.
    neumann_fun2 = bempp.api.GridFunction(neumann_space2, coefficients=soln_du2)
else:
    dirichlet_fun1 = soln[0] 
    neumann_fun1 = soln[1] 
    dirichlet_fun2 = soln[2] 
    neumann_fun2 = soln[3]  

Finally we calculate the solvation energy for the BEM-BEM-BEM coupling, using the equation of the potential at the position of the solute atoms.
\begin{equation}
\phi_m(x) =V_L\partial_{n}\phi_m^{\Gamma_{a}}(x)-K_L\phi_m^{\Gamma_{a}}(x)  \quad \quad
 \Delta G_{Sol}=\frac{1}{2}\sum_{i=1}^{n_{c}}Q_{i}\phi_{m}(x_i) 
\end{equation}   

In [12]:
#Result of the total solvation energy.
VF1 = bempp.api.operators.potential.laplace.single_layer(neumann_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(dirichl_space1, np.transpose(PC))
uF = VF1*neumann_fun1 - KF1*dirichlet_fun1 
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Solvation Energy: {:7.6f} [kCal/mol]'.format(E_Solv) )

#Total time.
end = time.time()
curr_time = (end - start)
print("Total time: {:5.2f} [s]".format(curr_time))

Solvation Energy: -71.152706 [kCal/mol]
Total time: 368.76 [s]
